In [1]:
import pandas as pd
import numpy as np
from scipy import stats
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import RFE
import plotly.express as px
import dash
from dash import dcc, html, Input, Output

# Wczytanie danych
sciezka_pliku = 'messy_data.csv'
dane = pd.read_csv(sciezka_pliku)

# Czyszczenie i przygotowanie danych
dane.columns = [col.strip() for col in dane.columns]  # Usunięcie spacji z nazw kolumn
dane = dane.drop_duplicates()
dane['clarity'] = dane['clarity'].str.lower()
dane['color'] = dane['color'].str.lower()
dane['cut'] = dane['cut'].str.lower()

kolumny_numeryczne = ['carat', 'x dimension', 'y dimension', 'z dimension', 'depth', 'table', 'price']
dane[kolumny_numeryczne] = dane[kolumny_numeryczne].apply(pd.to_numeric, errors='coerce')
dane = dane.dropna(subset=['carat'])

for kolumna in kolumny_numeryczne:
    if kolumna != 'carat':
        dane[kolumna].fillna(dane[kolumna].median(), inplace=True)

# Obsługa wartości odstających
z_wyniki = np.abs(stats.zscore(dane[kolumny_numeryczne]))
do_usuniecia = (z_wyniki >= 3).any(axis=1)
dane = dane[~do_usuniecia]

# Budowa modelu regresji
X = dane[['carat', 'x dimension', 'y dimension', 'z dimension', 'depth', 'table']]
y = dane['price']
X_treningowe, X_testowe, y_treningowe, y_testowe = train_test_split(X, y, test_size=0.2, random_state=0)
model = LinearRegression().fit(X_treningowe, y_treningowe)

selektor = RFE(model, n_features_to_select=4)
selektor.fit(X, y)
wybrane_cechy = []
for i in range(len(X.columns)):
    if selektor.support_[i]:
        wybrane_cechy.append(X.columns[i])

# Tworzenie dashboardu
aplikacja = dash.Dash(__name__)

aplikacja.layout = html.Div([
    html.H1("Analiza danych diamentów"),
    dcc.Graph(figure=px.histogram(dane, x='carat')),
    dcc.Graph(figure=px.scatter(dane, x='carat', y='price', color='clarity')),
    dcc.Graph(figure=px.scatter(x=y_testowe, y=model.predict(X_testowe), labels={'x': 'Cena rzeczywista', 'y': 'Cena przewidywana'})),
    html.Label('Wybierz zmienną:'),
    dcc.Dropdown(id='wybor-zmiennej', options=[{'label': cecha, 'value': cecha} for cecha in wybrane_cechy], value='carat'),
    dcc.Graph(id='wykres-zmiennej')
])

@aplikacja.callback(Output('wykres-zmiennej', 'figure'), Input('wybor-zmiennej', 'value'))
def aktualizuj_wykres(wybrana_zmienna):
    wykres = px.scatter(dane, x=wybrana_zmienna, y='price', color='clarity')
    return wykres

# Uruchomienie aplikacji dashboard
if __name__ == '__main__':
    aplikacja.run_server(debug=True)
